# Jupyter notebook based on ImageD11 to process 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 21/02/2025

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# desination of H5 files
# replace below with e.g.:
# dset_path = '/data/visitor/expt1234/20240101/PROCESSED_DATA/sample/dataset/sample_dataset.h5'

dset_path = ''

phase_str = 'Fe'

# path to parameters .json/.par
parfile = ''

# peak filtration options
cf_strong_frac = 0.999
cf_strong_dsmax = 1.017
cf_strong_dstol = 0.025

# indexing options
rings_to_use = [0, 1, 3]

# makemap options
symmetry = "cubic"
makemap_tol_seq = [0.02, 0.015, 0.01]

gridpars = {
        'DSTOL' : 0.004,
        'RING1'  : [1,0,],
        'RING2' : [0,],
        'NUL' : True,
        'FITPOS' : True,
        'tolangle' : 0.50,
        'toldist' : 100.,
        'NTHREAD' : 1 ,
}

grid_xlim = 600  # um - extent away from rotation axis to search for grains
grid_ylim = 600
grid_zlim = 200
grid_step = 100  # step size of search grid, um

# fraction of expected number of peaks to accept in Makemap output
frac = 0.85

# find the spike
absolute_minpks = 56

dset_prefix = 'ff'

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

# import utils
from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial
from ImageD11.peakselect import select_ring_peaks_by_intensity

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

sample = ds.sample
dataset = ds.dset
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# also set our parameters for indexing
ds.parfile = parfile
ds.save()

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# now let's select a phase to index from our parameters json

ucell = ds.phases.unitcells[phase_str]

print(ucell.lattice_parameters, ucell.spacegroup)

In [ ]:
# load 3d columnfile from disk

cf_3d = ds.get_cf_3d_from_disk()
ds.update_colfile_pars(cf_3d, phase_name=phase_str) 

cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
cf_3d.writefile(cf_3d_path)

In [ ]:
# plot the 3D peaks (fewer of them) as a cake (two-theta vs eta)
# if the parameters in the par file are good, these should look like straight lines

ucell.makerings(cf_3d.ds.max())

fig, ax = plt.subplots(figsize=(16,9), layout='constrained')

ax.scatter(cf_3d.ds, cf_3d.eta, s=1)
ax.vlines(ucell.ringds, -50, 50, zorder=0, color='red')

ax.set_xlabel("D-star")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# here we are filtering our peaks (cf_3d) to select only the strongest ones for indexing purposes only!
# dsmax is being set to limit rings given to the indexer - 6-8 rings is normally good

# USER: modify the "frac" parameter below and re-run the cell until the orange dot sits nicely on the "elbow" of the blue line
# this indicates the fractional intensity cutoff we will select
# if the blue line does not look elbow-shaped in the logscale plot, try changing the "doplot" parameter (the y scale of the logscale plot) until it does

cf_strong = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, doplot=0.5, dstol=cf_strong_dstol)
print(f"Got {cf_strong.nrows} strong peaks for indexing")

In [ ]:
# we will also export some additional strong peaks across all rings
# this will be useful for grain refinement later (using makemap)

cf_strong_allrings = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_3d.ds.max(), doplot=0.5, dstol=cf_strong_dstol)
print(f"Got {cf_strong_allrings.nrows} strong peaks for makemap")
cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
cf_strong_allrings.writefile(cf_strong_allrings_path)

In [ ]:
# now we can take a look at the intensities of the remaining peaks

fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)

ax.plot(cf_3d.ds, cf_3d.sum_intensity,',', label='cf_3d')
ax.plot(cf_strong.ds, cf_strong.sum_intensity,',', label='cf_strong')
ax.vlines(ucell.ringds, 1e3, 1e4, zorder=0, color='red')

ax.semilogy()

ax.set_xlabel("Dstar")
ax.set_ylabel("Intensity")
ax.legend()

plt.show()

In [ ]:
# specify our ImageD11 indexer with these peaks

indexer = ImageD11.indexing.indexer_from_colfile_and_ucell(cf_strong, ucell)

print(f"Indexing {cf_strong.nrows} peaks")

# USER: set a tolerance in d-space (for assigning peaks to powder rings)
indexer.ds_tol = cf_strong_dstol

# change the log level so we can see what the ring assigments look like

ImageD11.indexing.loglevel = 1

# assign peaks to powder rings

indexer.assigntorings()

# change log level back again

ImageD11.indexing.loglevel = 3

In [ ]:
# let's plot the assigned peaks

fig, ax = plt.subplots()

# indexer.ra is the ring assignments

ax.scatter(indexer.colfile.ds, indexer.colfile.eta, c=indexer.ra%20, cmap='tab20', s=1)
ax.vlines(ucell.ringds, -50, 50, zorder=0, color='red')
ax.set_xlabel("d-star")
ax.set_ylabel("eta")
ax.set_xlim(min(ucell.ringds[0], cf_strong.ds.min()) - 0.02, cf_strong.ds.max() + 0.02)
plt.show()

In [ ]:
# now we need to decide which rings to use in the grid index
# typically, 3-4 low multiplicity rings are good

mask = np.zeros(cf_strong.nrows, dtype=bool)

for ring in rings_to_use:
    mask |= indexer.ra == ring

peaks_to_export = cf_strong.copy()
peaks_to_export.filter(mask)

In [ ]:
# now we can take a look at the intensities of the peaks we will be exporting

fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)

ax.plot(cf_3d.ds, cf_3d.sum_intensity,',', label='cf_3d')
ax.plot(peaks_to_export.ds, peaks_to_export.sum_intensity,',', label='peaks to export')
ax.vlines(ucell.ringds, 1e3, 1e4, zorder=0, color='red')
ax.semilogy()

ax.set_xlabel("Dstar")
ax.set_ylabel("Intensity")
ax.legend()

plt.show()

In [ ]:
grid_peaks_path = f'{sample}_{dataset}_3d_peaks_grid.flt'
peaks_to_export.writefile(grid_peaks_path)

In [ ]:
omega_slop = ds.ostep/2

In [ ]:
# now we need to compute the number of expected peaks
# to do this, you add up the multiplicites of the rings you chose
# if you recorded a 360 degree scan, multiply the result by 2
# e.g given this output:

# info: Ring     (  h,  k,  l) Mult  total indexed to_index  ubis  peaks_per_ubi   tth
# info: Ring 3   ( -2, -2,  0)   12   2251       0     2251    93     24  16.11
# info: Ring 2   ( -1, -1, -2)   24   4899       0     4899   101     48  13.94
# info: Ring 1   ( -2,  0,  0)    6   1233       0     1233   102     12  11.37
# info: Ring 0   ( -1, -1,  0)   12   2861       0     2861   118     24  8.03

# selecting rings 0,1,3 we would get
# we would get (12+6+12)*2 = 84 peaks

In [ ]:
peaks_expected = (12+6+12)*2

# choose the fraction of the number of peaks expected - this should be around 0.9 if you had a good clean segementation
# if you suspect you are missing peaks in your data, decrease to around 0.6

minpeaks = int(np.round(peaks_expected * frac, 2))
minpeaks

In [ ]:
nproc = len(os.sched_getaffinity(os.getpid())) - 3

In [ ]:
# now we write a classic parameter file for makemap.py
oldparfile = phase_str + '.par'
ds.phases.to_old_pars_file(oldparfile, phase_str)

In [ ]:
from ImageD11.grid_index_parallel import grid_index_parallel

gridpars['COSTOL'] = np.cos(np.radians(90 - ds.ostep))
gridpars['NPROC'] = nproc
gridpars['NPKS'] = minpeaks
gridpars['OMEGAFLOAT'] = omega_slop
gridpars['TOLSEQ'] = makemap_tol_seq
gridpars['SYMMETRY'] = symmetry

translations = [(t_x, t_y, t_z) # grid to search
    for t_x in range(-grid_xlim, grid_xlim+1, grid_step)
    for t_y in range(-grid_ylim, grid_ylim+1, grid_step) 
    for t_z in range(-grid_zlim, grid_zlim+1, grid_step) ]

import random
random.seed(42) # reproducible
random.shuffle(translations)

tmp_output_path = 'tmp'
map_path = 'alltmp.map'

grid_index_parallel(grid_peaks_path, oldparfile, tmp_output_path, gridpars, translations)

In [ ]:
# import our initially indexed grains

grains2 = ImageD11.grain.read_grain_file(map_path)

for g in grains2:
    g.ref_unitcell = ucell

utils.get_rgbs_for_grains(grains2)

In [ ]:
# inverse pole figure of grain orientations

utils.plot_all_ipfs(grains2)

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains2, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
# 3D scatter plot of grain positions coloured by inverse pole figure orientation

utils.plot_grain_positions(grains2, colour='z', centre_plot=False, size_scaling=0.5)

In [ ]:
# run makemap against the selected grid peaks

new_map_path = f'alltmp.map.new'
new_grid_peaks_path = f'{sample}_{dataset}_3d_peaks_grid.flt.new'

makemap_output = !makemap.py -p {oldparfile} -u {map_path} -U {new_map_path} -f {grid_peaks_path} -s {symmetry} -t {makemap_tol_seq[-1]} --omega_slop={omega_slop} --no_sort

In [ ]:
utils.plot_grain_histograms(new_grid_peaks_path, new_map_path, oldparfile, omega_slop, tol=makemap_tol_seq[-1])

In [ ]:
# re-import our refined grains from the makemap procedure

grains3 = ImageD11.grain.read_grain_file(new_map_path)

In [ ]:
# remove grains with no peaks

grains3 = [grain for grain in grains3 if "no peaks" not in grain.intensity_info]

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains3, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in grains3], bins=50)
# ax.semilogy()
plt.show()

In [ ]:
# filter out grains with fewer than absolute_minpks peaks
grains_filtered = [grain for grain in grains3 if float(grain.npks) > absolute_minpks]

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains_filtered, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
for g in grains_filtered:
    g.intensity = float(g.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))

In [ ]:
# write the filtered grains to disk

filtered_map_path = f'{sample}_{dataset}_nice_grains.map'

ImageD11.grain.write_grain_file(filtered_map_path, grains_filtered)

In [ ]:
# run makemap again against all peaks

new_filtered_map_path = f'{sample}_{dataset}_nice_grains.map.new'
new_cf_3d_path = cf_3d_path + '.new'

makemap_output = !makemap.py -p {oldparfile} -u {filtered_map_path} -U {new_filtered_map_path} -f {cf_3d_path} -s {symmetry} -t {makemap_tol_seq[-1]} --omega_slop={omega_slop} --no_sort

In [ ]:
grains_final = ImageD11.grain.read_grain_file(new_filtered_map_path)

for g in grains_final:
    g.ref_unitcell = ucell

utils.get_rgbs_for_grains(grains_final)

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains_final, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains_final, colour='z', centre_plot=False, size_scaling=0.5)

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in grains_final], bins=50)
# ax.semilogy()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter([float(grain.npks) for grain in grains_final], [float(g.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", "")) for g in grains_final])
ax.set_xlabel('npks')
ax.set_ylabel('sum_int')
ax.semilogy()
plt.show()

In [ ]:
len(grains_final)

In [ ]:
utils.plot_grain_histograms(new_cf_3d_path, new_filtered_map_path, oldparfile, omega_slop, tol=makemap_tol_seq[-1])

In [ ]:
unit_cell_lengths = np.array([np.sort(g.unitcell.copy()[:3]) for g in grains_final])
median_unit_cell = np.median(unit_cell_lengths, axis=0)

fig, ax = plt.subplots(constrained_layout=True)
ax.plot(unit_cell_lengths)
ax.hlines(median_unit_cell, 0, len(unit_cell_lengths))
ax.set_xlabel("Grain ID")
ax.set_ylabel("Unit cell length")
plt.show()

In [ ]:
# import makemap output columnfile with peak assignments
cf_3d = ImageD11.columnfile.columnfile(new_cf_3d_path)

# write 3D columnfile to disk
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile, name='peaks')

In [ ]:
ds.save_grains_to_disk(grains_final, phase_name=phase_str)

In [ ]:
ds.save()

In [ ]:
# cleaning up

for path in [
    cf_3d_path,
    cf_strong_allrings_path,
    grid_peaks_path,
    tmp_output_path + '.flt',
    map_path,
    new_map_path,
    new_grid_peaks_path,
    filtered_map_path,
    new_filtered_map_path,
    new_cf_3d_path,
]:
    if os.path.exists(path):
        os.remove(path)